In [11]:
import os
import MDAnalysis as md
from MDAnalysis.analysis import align
import h5py
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
# Here we select all the amino acid residues that shall be aligned to the ubiWT. This is in the same fashion as performed by PyMoL
selection_WT = 'all'
selection_6  = "(resid 1:5, 1:53, 55:72, 74)"
selection_20 = "(resid 1:6, 10:19, 21:46, 48:71)"
selection_35 = "(resid 1:6, 11:34, 36:71)"
selection_48 = "(resid 1:1, 3:6, 11:20, 22:45, 49:71)"

In [ ]:
# For all energies ...
for energy in ['300ev', '600ev', '2000ev']:

    # for all mutants ...
    for mutant in [['ubiWT', selection_WT], ['ubi6', selection_6], ['ubi20', selection_20], ['ubi35', selection_35], ['ubi48', selection_48]]:
        conf = mutant[0]
        cpath = f"/home/simon/results/quick_pulse/{energy}"
        os.chdir(cpath)
        os.mkdir(f"{conf}_aligned")

        # for all simulations ...
        for i in range(100):

            os.chdir(cpath)

            # load the trajectory and structure of the ubiWT (which will be aligned TO)
            trr_WT = f"/home/simon/results/last_frame/{energy}/ubiWT_static/sim1/ubiWT.trr"
            gro_WT = f"/home/simon/structure_files/gros/ubiWT.gro"

            # load the current trajectory and structure (that will be aligned to ubiWT)
            gro = f"/home/simon/structure_files/gros/{conf}.gro"
            trr = f"{cpath}/{conf}_unaligned/sim{i+1}/{conf}.trr"

            # Calculate the md.analysis universes
            structure_WT = md.Universe(gro_WT, trr_WT,)
            structure_conf = md.Universe(gro, trr,)

            # Since we will align the first frame, we select it
            structure_WT.trajectory[0]
            structure_conf.trajectory[0]

            # take this mutants selection of amino acids to be aligned
            selection = mutant[1]

            # Align the structure to the ubiWT
            align.AlignTraj(structure_conf, structure_WT, select=selection, in_memory=True).run()
            ag = structure_conf.atoms.select_atoms("all")

            # Here we calculate the new data for the data.h5 file
            structure_conf.trajectory[0]
            vel_i = ag.velocities.copy()  # Copy initial velocities
            pos_i = ag.positions.copy()  # Copy initial positions
            structure_conf.trajectory[-1]  # Move to the last frame of the trajectory
            vel_f = ag.velocities.copy()  # Copy final velocities
            pos_f = ag.positions.copy()  # Copy final positions
            vel_data = [(x / np.linalg.norm(x)) if np.linalg.norm(x) != 0 else x for x in vel_f]  # Normalize velocities
            pos_data = [(x / np.linalg.norm(x)) if np.linalg.norm(x) != 0 else x for x in (pos_f - pos_i)]  # Normalize displacements
            
            os.chdir(f"{conf}_aligned")

            aligned_trr_path = f"sim{i+1}"
            os.makedirs(aligned_trr_path)

            # Now we create the h5-file
            with md.Writer(f"{aligned_trr_path}/{conf}.trr", structure_conf.atoms.n_atoms) as w:
                for ts in structure_conf.trajectory:
                    w.write(structure_conf.atoms)

            # Then we open the data.h5 file and write all the information
            with h5py.File(f"data.h5", 'a') as file:
                group_path = f"sim{i+1}"
                group = file.require_group(group_path)  # Create or get the group for this simulation
                group.create_dataset("unit_velocity", data=vel_data)
                group.create_dataset("unit_displacement", data=pos_data)
                group.create_dataset("initial_position", data=pos_i)
                group.create_dataset("final_position", data=pos_f)
                group.create_dataset("initial_velocity", data=vel_i)
                group.create_dataset("final_velocity", data=vel_f)

/home/simon/.conda/envs/explode/lib/python3.8/site-packages/MDAnalysis/analysis/align.py:1394: SelectionWarning: Reference and trajectory atom selections do not contain the same number of atoms: 
atoms:    N_ref=1030, N_traj=1029
but we attempt to create a valid selection (use strict=True to disable this heuristic).
  warnings.warn(msg, category=SelectionWarning)
